<a href="https://colab.research.google.com/github/ollihansen90/linclassifiers/blob/main/Futureskills/LinClass_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kapitel 8 - Perzeptron

## Setup

In [ ]:
# Todo: utils-Datei anlegen, die am Anfang geladen wird
# utils_linclass.py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from IPython.display import display
from PIL import Image
from IPython.display import Image as Image_

class Koordinatensystem():
    def __init__(self, config={}):
        self.config = {
            "size": 11,
            "grid_alpha": 0.1,
            }
        self.config.update(config)

    def draw(self):
        size = self.config["size"]
        grid_alpha = self.config["grid_alpha"]
        plt.plot([0,0], [-size,size], "k")
        plt.plot([-size,size], [0,0], "k")
        for t in range(1,size):
            plt.plot([t,t], [-size,size], "k", alpha=grid_alpha)
            plt.plot([-t,-t], [-size,size], "k", alpha=grid_alpha)
            plt.plot([-size,size], [t,t], "k", alpha=grid_alpha)
            plt.plot([-size,size], [-t,-t], "k", alpha=grid_alpha)
            if t%2==0:
                plt.plot([t,t], [-0.1,0.1], "k")
                plt.text(t,-0.5, str(t), {"horizontalalignment":"center", "verticalalignment":"center"})
                plt.plot([-t,-t], [-0.1,0.1], "k")
                plt.text(-t,-0.5, str(-t), {"horizontalalignment":"center", "verticalalignment":"center"})
                plt.plot([-0.1,0.1], [t,t], "k")
                plt.text(-0.4, -t, str(-t), {"horizontalalignment":"right", "verticalalignment":"center"})
                plt.plot([-0.1,0.1], [-t,-t], "k")
                plt.text(-0.4, t, str(t), {"horizontalalignment":"right", "verticalalignment":"center"})
        plt.plot([-0.1,0,0.1], [size-0.2,size,(size-0.2)], "k")
        plt.plot([(size-0.2),size,(size-0.2)], [-0.1,0,0.1], "k")
        plt.plot([-0.1,0,0.1], [-(size-0.2),-size,-(size-0.2)], "k")
        plt.plot([-(size-0.2),-size,-(size-0.2)], [-0.1,0,0.1], "k")
        plt.text(-0.4, (size-0.2), "y", {"horizontalalignment":"center", "verticalalignment":"center"})
        plt.text(size, -0.4, "x", {"horizontalalignment":"center", "verticalalignment":"center"})

class Vektorfolge():
    def __init__(self, veclist, colorlist=list(mcolors.TABLEAU_COLORS.keys()), alphalist=None):
        self.veclist = veclist
        self.colorlist = colorlist
        self.alphalist = [1]*len(veclist) if alphalist==None else alphalist
        self.config = {"angles":'xy', "scale_units":'xy', "scale":1, "width":0.005, "zorder":2}

    def draw(self):
        plt.quiver(0,0,*self.veclist[0],**self.config, color=self.colorlist[0], alpha=self.alphalist[0])
        v = self.veclist[0].copy()
        for i in range(1, len(self.veclist)):
            plt.quiver(
                *v,
                *self.veclist[(i)],
                **self.config,
                color=self.colorlist[(i)%len(self.colorlist)],
                alpha=self.alphalist[i]
                )
            v += self.veclist[i].copy()

class Gerade():
    def __init__(self, aufpunkt, richtung):
        assert aufpunkt.shape==np.zeros(2).shape, "Der Vektor für den Aufpunkt hat nicht die richtige Form."
        assert richtung.shape==np.zeros(2).shape, "Der Vektor für die Richtung hat nicht die richtige Form."
        assert np.sum(np.abs(richtung))>0, "Der Vektor darf nicht [0,0] sein."
        self.aufpunkt = aufpunkt
        self.richtung = richtung

    def draw(self, col="tab:blue"):
        size = 11
        ecke1 = self.aufpunkt.copy()
        while np.max(np.abs(ecke1))<=size:
            ecke1 += self.richtung
        ecke2 = self.aufpunkt.copy()
        while np.max(np.abs(ecke2))<=size:
            ecke2 -= self.richtung
        ecken = np.stack([ecke1, ecke2])
        plt.plot(ecken[:,0], ecken[:,1], col)

    def on_gerade(self, pt):
        p = pt.copy().astype(float)
        p -= self.aufpunkt
        if np.sum(p**2)==0:
            return True
        p /= np.linalg.norm(p)
        return np.abs((self.richtung/np.linalg.norm(self.richtung))@p)>1-1e-6

def generate_data():
    np.random.seed(1)

    N = 100
    data = 3*np.random.randn(N,2)
    data[:N//2] += np.array([-4,6])
    data[N//2:] += np.array([2,-8])
    data = np.column_stack([data, -np.ones(N)])
    label = np.array(N//2*[1]+N//2*[-1])

    # Permutiere Daten
    perm = np.random.permutation(len(label))
    data = data[perm]
    label = label[perm]

    return data, label

def show_vid(vid):
    every = max((len(vid)//50, 1))
    vid = np.stack(vid[::every])
    vid = [Image.fromarray(img) for img in vid]
    vid[0].save("array.gif", save_all=True, append_images=vid[1:], duration=50, loop=0)

    with open('/content/array.gif','rb') as f:
        display(Image_(data=f.read(), format='gif'))

In [ ]:
# Todo: utils-Datei anlegen, die am Anfang geladen wird
# utils_linclass_08.py
import numpy as np
# from utils import Vektorfolge, Koordinatensystem, Gerade

def draw(w, data, label):
    size = 11
    config = {"size": size}
    koordinatensystem = Koordinatensystem(config)
    gewichtsvektor = w[:2]
    theta = w[-1]
    if w[1]==0:
        aufpunkt = np.array([(theta-gewichtsvektor[1])/gewichtsvektor[0],1])
    else:
        aufpunkt = np.array([1,(theta-gewichtsvektor[0])/gewichtsvektor[1]])
    gerade = Gerade(aufpunkt, gewichtsvektor[::-1]*np.array([1,-1]))
    vec = Vektorfolge([aufpunkt, gewichtsvektor], ["blue", "tab:green"], alphalist=[0,1])

    fig = plt.figure(figsize=[7,7])
    koordinatensystem.draw()
    gerade.draw(col="r")
    vec.draw()
    plt.scatter(data[label==1,0], data[label==1,1])
    plt.scatter(data[label==-1,0], data[label==-1,1])
    plt.axis("equal");plt.xlim([-size,size]);plt.ylim([-size,size]);plt.axis("off")
    fig.canvas.draw()
    image_flat = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image_flat.reshape(*reversed(fig.canvas.get_width_height()), 3)
    plt.close()
    return image


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import trange

In diesem Notebook soll der Lernschritt eines Perzeptrons implementiert werden. Wie im Video bereits erklärt, ist ein Perzeptron ein linearer Klassifizierer mit einer Aktivierungsfunktion, die für eine beliebige (reelle) Eingabe das Vorzeichen zurückgibt.

## Daten generieren
Die Daten werden genau wie die in Aufgabe 2 von Kapitel 7 generiert. Es handelt sich also um zwei Punktewolke, die sich linear trennen lassen. Die blauen Punkte haben Label `+1`, die orangenen Punkte haben Label `-1`. Beispielhaft wird ebenso eine Gerade eingezeichnet, die das Klassifikationsproblem (noch) nicht löst.

In [ ]:
data, label = generate_data()
w = np.array([1,1,-1])

plt.figure(figsize=[7,7])
plt.imshow(draw(w, data, label))
plt.axis("off")
plt.show()

## Update-Schritt
Wie im Video bereits erklärt, kann die Klassifikationsgerade schrittweise mit Hilfe der Perzeptron-Lernregel angepasst werden. Hierbei werden die Gewichte, angepasst mit dem Threshold-Trick (also erweitert um `-1`) folgendermaßen aktualisiert: $w_{t+1} = w_t+\Delta w_t$ mit $\Delta w_t= \epsilon\cdot(s^\mu-y)\cdot x$. Die einzelnen Parameter in der Formel haben folgende Bedeutung:
- $\epsilon$: Lernrate, im Code `lr`
- $s^\mu$: Ausgabe des Perzeptrons (also `+1` oder `-1`), im Code `out`
- $y$: Label von Punkt $x$ (`+1` oder `-1`), im Code `label`
- $x$: Punkt, der in das Perzeptron gegeben wird, im Code `data`.

Da es sich bei `data` und `label` um lange Listen handelt, werden diese eintragsweise verarbeitet, indem ein Index durch die Liste läuft. Der `i`-te Eintrag wäre hier `data[i]`, bzw. `label[i]`.

Die Aktivierungsfunktion, die das Vorzeichen einer Eingabe zurückgibt, wurde über eine `if`-Abfrage implementiert.

Sind Label und Ausgabe des Perzeptrons gleich, so gibt es keine Veränderung der Gewichte, da $\Delta w_t= \epsilon\cdot(1-1)\cdot x=0$ bzw. $\Delta w_t= \epsilon\cdot(-1-(-1))\cdot x=0$ gilt.

In [ ]:
datal, label = generate_data()

# --- Gewichte und Lernrate anpassen ---
w = np.array(
        [4,-4,-1]
    )
lr = 0.001
# -------------------------------------

vid = [draw(w, data, label)]

for i in trange(len(data)):
    out = data[i]@w # Skalarprodukt des Datenpunktes mit dem Gewichtsvektor

    # Vorzeichenfunktion
    if out>=0:
        out = 1
    else:
        out = -1

    # Berechne Updateschritt
    dw = lr*(label[i]-out)*data[i]
    # Wende Updateschritt an
    w = w+dw

    # Überspringe alle Bilder, in denen sich nichts ändert
    if out == label[i]:
        continue
    image = draw(w, data, label)
    vid.append(image)

show_vid(vid)

## Update über Mehrere Epochen
Üblicherweise wird ein Perzeptron über mehrere Epochen trainiert, da das Klassifikationsproblem oftmals nach nur einer Epoche nicht gelöst ist. "Epoche" ist hier so definiert, dass jeder Datenpunkt einmal betrachtet wurde und entsprechend ein Updateschritt durchgeführt wurde.

Die Funktion `update_epoch` führt den Updateschritt für eine Epoche durch, ohne die Abbildungen für jeden Updateschritt zu erstellen.

In [ ]:
def update_epoch(w, lr, data, label):
    # Durchlaufe den Index jedes Datenpunktes
    for i in range(len(data)):
        out = data[i]@w # Skalarprodukt des Datenpunktes mit dem Gewichtsvektor

        # Vorzeichenfunktion
        if out>=0:
            out = 1
        else:
            out = -1

        # Berechne Updateschritt
        dw = lr*(label[i]-out)*data[i]
        # Wende Updateschritt an
        w = w+dw
    return w

## Perzeptron-Training
Im letzten Abschnitt soll nun alles zusammengesetzt werden: Ein Perzeptron wird definiert, das das gegebene Klassifikationsproblem nicht löst. Schrittweise werden die Gewichte jetzt so angepasst, dass ein Klassifikator entsteht, der die beiden Klassen korrekt voneinander trennt.

Im unteren Abschnitt können Hyperparameter festgelegt werden. Diese Hyperparameter sind die *Lernrate* `lr`, eine *Toleranz* `tol`, sowie eine *maximale Anzahl an Schritten* `n_max`.

Die Toleranz und die Maximalzahl an Schritten definieren eine Abbruchbedingung des Trainings. Nur in den seltensten Fällen gibt es Problemstellungen, die linear trennbar sind. Wie oben beobachtet werden konnte, gibt es Updateschritte, solange es noch falsche Klassifikationen gibt. Sind die Klassen nicht linear trennbar, so würde das Training ewig weitergeführt werden.

Die einfachste Abbruchbedingung ist eine maximale Anzahl an Schritten. Sobald diese Zahl erreicht wird, soll das Training beendet werden. Hierbei ist es unerheblich, wie gut das Problem zu dem Zeitpunkt gelöst ist.

Eine weitere Abbruchbedingung ist die Toleranz. Die Toleranz kann sehr unterschiedlich definiert sein. Im folgenden Codeblock ist sie so definiert, dass das Training abgebrochen wird, sobald sich die Gewichte von einer Epoche zur nächsten nicht mehr stark ändern.

In [ ]:
def run(lr,n_max=50, tol=1e-5):
    # Generiere Daten
    data, label = generate_data()

    # Initialisiere Gewichte w
    w = np.array([1,-1,1])

    # Initialisiere Video mit dem Startbild
    vid = [draw(w, data, label)]

    # Training
    for epoch in trange(n_max):
        w_alt = w.copy()

        # Update der Gewichte
        w = update_epoch(w, lr, data, label)

        # Einzeichnen der Punkte und des Perzeptrons
        image = draw(w, data, label)
        vid.append(image)

        # Abbruch bei wenig Veränderung
        if np.linalg.norm(w-w_alt)<tol:
            break
    show_vid(vid)

# --- Hier können die Hyperparameter erstellt werden ---
lernrate = 0.0001
tol = 0.00001
n_max = 50
# ---------------------------------------------------------

run(lr=lernrate, n_max=n_max, tol=tol)